In [1]:
import copy
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

def progn(*args):
    for arg in args:
        arg()

def prog2(out1, out2): 
    return partial(progn,out1,out2)

def prog3(out1, out2, out3):     
    return partial(progn,out1,out2,out3)

def if_then_else(condition, out1, out2):
    out1() if condition() else out2()

class AntSimulator(object):
    direction = ["north","east","south","west"]
    dir_row = [1, 0, -1, 0]
    dir_col = [0, 1, 0, -1]
    
    def __init__(self, max_moves):
        self.max_moves = max_moves
        self.moves = 0
        self.eaten = 0
        self.routine = None
        
    def _reset(self):
        self.row = self.row_start 
        self.col = self.col_start 
        self.dir = 1
        self.moves = 0  
        self.eaten = 0
        self.matrix_exc = copy.deepcopy(self.matrix)

    @property
    def position(self):
        return (self.row, self.col, self.direction[self.dir])
            
    def turn_left(self): 
        if self.moves < self.max_moves:
            self.moves += 1
            self.dir = (self.dir - 1) % 4

    def turn_right(self):
        if self.moves < self.max_moves:
            self.moves += 1    
            self.dir = (self.dir + 1) % 4
        
    def move_forward(self):
        if self.moves < self.max_moves:
            self.moves += 1
            self.row = (self.row + self.dir_row[self.dir]) % self.matrix_row
            self.col = (self.col + self.dir_col[self.dir]) % self.matrix_col
            if self.matrix_exc[self.row][self.col] == "food":
                self.eaten += 1
            self.matrix_exc[self.row][self.col] = "passed"

    def sense_food(self):
        ahead_row = (self.row + self.dir_row[self.dir]) % self.matrix_row
        ahead_col = (self.col + self.dir_col[self.dir]) % self.matrix_col        
        return self.matrix_exc[ahead_row][ahead_col] == "food"
   
    def if_food_ahead(self, out1, out2):
        return partial(if_then_else, self.sense_food, out1, out2)
   
    def run(self,routine):
        self._reset()
        while self.moves < self.max_moves:
            routine()
    
    def parse_matrix(self, matrix):
        self.matrix = list()
        for i, line in enumerate(matrix):
            self.matrix.append(list())
            for j, col in enumerate(line):
                if col == "#":
                    self.matrix[-1].append("food")
                elif col == ".":
                    self.matrix[-1].append("empty")
                elif col == "S":
                    self.matrix[-1].append("empty")
                    self.row_start = self.row = i
                    self.col_start = self.col = j
                    self.dir = 1
        self.matrix_row = len(self.matrix)
        self.matrix_col = len(self.matrix[0])
        self.matrix_exc = copy.deepcopy(self.matrix)

ant = AntSimulator(600)

pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(ant.if_food_ahead, 2)
pset.addPrimitive(prog2, 2)
pset.addPrimitive(prog3, 3)
pset.addTerminal(ant.move_forward)
pset.addTerminal(ant.turn_left)
pset.addTerminal(ant.turn_right)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("expr_init", gp.genFull, pset=pset, min_=1, max_=2)

# Structure initializers
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr_init)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalArtificialAnt(individual):
    # Transform the tree expression to functionnal Python code
    routine = gp.compile(individual, pset)
    # Run the generated routine
    ant.run(routine)
    return ant.eaten,

toolbox.register("evaluate", evalArtificialAnt)
toolbox.register("select", tools.selTournament, tournsize=7)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

def main():
    # random.seed(69)
    
    # there are 89 bits of food...
    with  open("ant_santafe_trail.txt") as trail_file:
      ant.parse_matrix(trail_file)
    
    pop = toolbox.population(n=800)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 50, stats, halloffame=hof)
    
    return pop, hof, stats


In [2]:
pop, hof, stats = main()


gen	nevals	avg	std    	min	max
0  	800   	1.8	3.34739	0  	28 
1  	507   	5.1325	4.86518	0  	28 
2  	469   	8.6225	6.5892 	0  	31 
3  	495   	11.6163	9.04193	0  	44 
4  	480   	15.685 	11.708 	0  	44 
5  	457   	20.1513	13.3115	0  	44 
6  	477   	23.24  	15.4595	0  	50 
7  	463   	26.8738	16.8297	0  	50 
8  	459   	27.6788	18.5668	0  	50 
9  	474   	29.6962	18.5059	0  	50 
10 	479   	29.8675	19.5672	0  	50 
11 	465   	31.5138	20.2697	0  	50 
12 	466   	32.985 	21.0249	0  	50 
13 	464   	33.9637	20.8809	0  	50 
14 	483   	33.8588	20.6262	0  	50 
15 	486   	33.33  	21.1036	0  	52 
16 	467   	33.7   	20.8723	0  	52 
17 	482   	33.46  	20.9215	0  	53 
18 	479   	35.465 	20.743 	0  	53 
19 	513   	37.4625	20.6521	0  	65 
20 	471   	39.7388	20.3535	0  	65 
21 	515   	40.835 	21.0385	0  	65 
22 	492   	42.8563	22.7152	0  	65 
23 	481   	48.5138	22.5853	0  	67 
24 	502   	50.2188	23.1774	0  	67 
25 	455   	53.18  	21.0495	0  	67 
26 	474   	52.2587	22.1085	0  	67 
27 	466   	51.405 	22.5031	0  

In [21]:
stack = [1]
for node in hof[0]:
    print("  " * len(stack) + str(node.name))# + str(stack))
    if isinstance(node, gp.Primitive):
        stack.append(node.arity)
    while stack[-1] == 0:
        del stack[-1]
    stack[-1] -= 1


  prog3
    prog3
      move_forward
      if_food_ahead
        prog2
          move_forward
          move_forward
        turn_right
      move_forward
    prog2
      move_forward
      move_forward
    if_food_ahead
      prog2
        if_food_ahead
          if_food_ahead
            move_forward
            turn_right
          move_forward
        prog3
          move_forward
          turn_left
          turn_right
      turn_left
